In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

hospital_df = pd.read_csv('../downloads/normalizacion/hospital.csv', delimiter=',')
hospital_activity_df = pd.read_csv('../downloads/normalizacion/hospital_activity.csv', delimiter=';')
hospital_municipality_df = pd.read_csv('../downloads/normalizacion/hospital_municipality.csv', delimiter=';')
hospital_resources_df = pd.read_csv('../downloads/normalizacion/hospital_resources.csv', delimiter=';')
municipality_df = pd.read_csv('../downloads/normalizacion/municipality.csv', delimiter=',')
municipality_demographics_df = pd.read_csv('../downloads/normalizacion/municipality_demographics.csv', delimiter=',')

print('Hospitales:')
display(hospital_df.head())
print('Actividad hospitalaria:')
display(hospital_activity_df.head())
print('Hospital-Municipio:')
display(hospital_municipality_df.head())
print('Recursos hospitalarios:')
display(hospital_resources_df.head())
print('Municipios:')
display(municipality_df.head())
print('Demografía municipios:')
display(municipality_demographics_df.head())

# Filtro solicitado sobre hospital_municipality_df
exclude_hospital_id = 'hospital_central_de_la_defensa_gomez_ulla'
exclude_municipality_names = ['Moratalaz', 'Vicálvaro y Retiro', 'Braojos de la Sierra', 'nan']

# Convertir municipality_id a string para evitar problemas de comparación
hospital_municipality_df['municipality_id'] = hospital_municipality_df['municipality_id'].astype(str)

# Filtrar los municipality_id nulos y vacíos
hospital_municipality_df = hospital_municipality_df[hospital_municipality_df['municipality_id'].notnull() & (hospital_municipality_df['municipality_id'] != '')]

# Filtrar por municipality_id
hospital_municipality_df = hospital_municipality_df[~hospital_municipality_df['municipality_id'].isin(exclude_municipality_names)]

# Filtrar por hospital_id
hospital_municipality_df = hospital_municipality_df[hospital_municipality_df['hospital_id'] != exclude_hospital_id]

hospital_resources_2022 = hospital_resources_df[hospital_resources_df['year'].astype(str) == '2022'].copy()
hospital_resources_2022['hospital_id'] = hospital_resources_2022['hospital_id'].astype(str)

hospital_activity_2022 = hospital_activity_df[hospital_activity_df['year'].astype(str) == '2022'].copy()
hospital_activity_2022['hospital_id'] = hospital_activity_2022['hospital_id'].astype(str)

municipality_demographics_2022 = municipality_demographics_df[municipality_demographics_df['year'].astype(str) == '2022'].copy()
municipality_demographics_2022['id_secondary_municipality'] = municipality_demographics_2022['id_secondary_municipality'].astype(str)

# === CÁLCULO DEL NIVEL DE SERVICIO DE HOSPITALES ===

print("\n=== ANÁLISIS DEL NIVEL DE SERVICIO HOSPITALARIO ===\n")

# 1. PREPARAR DATOS DE RECURSOS POR HOSPITAL
hospital_resources_pivot = hospital_resources_2022.pivot_table(
    index='hospital_id',
    columns='type_resources',
    values='total',
    fill_value=0
).reset_index()

print('Recursos hospitalarios por hospital (formato pivote):')
display(hospital_resources_pivot.head())

# 2. PREPARAR DATOS DE ACTIVIDAD POR HOSPITAL
hospital_activity_pivot = hospital_activity_2022.pivot_table(
    index='hospital_id',
    columns='type_activity',
    values='total',
    fill_value=0
).reset_index()

print('\nActividad hospitalaria por hospital (formato pivote):')
display(hospital_activity_pivot.head())

# 3. UNIR RECURSOS Y ACTIVIDAD POR HOSPITAL
hospital_service_df = hospital_resources_pivot.merge(hospital_activity_pivot, on='hospital_id', how='outer')

# 4. CALCULAR INDICADORES DE NIVEL DE SERVICIO

# 4.1 Tasa de ocupación de camas (días de hospitalización / (camas * 365))
if 'Camas instaladas' in hospital_service_df.columns and 'Total ingresos' in hospital_service_df.columns and 'Estancia media global' in hospital_service_df.columns:
    hospital_service_df['dias_hospitalizacion'] = hospital_service_df['Total ingresos'] * hospital_service_df['Estancia media global']
    hospital_service_df['capacidad_anual_camas'] = hospital_service_df['Camas instaladas'] * 365
    hospital_service_df['tasa_ocupacion_camas'] = np.where(
        hospital_service_df['capacidad_anual_camas'] > 0,
        hospital_service_df['dias_hospitalizacion'] / hospital_service_df['capacidad_anual_camas'],
        0
    )

# 4.2 Eficiencia en urgencias (% de ingresos desde urgencias)
if 'Urgencias totales' in hospital_service_df.columns and 'Total ingresos' in hospital_service_df.columns:
    hospital_service_df['tasa_ingreso_urgencias'] = np.where(
        hospital_service_df['Urgencias totales'] > 0,
        hospital_service_df['Total ingresos'] / hospital_service_df['Urgencias totales'],
        0
    )

# 4.3 Eficiencia de ingresos urgentes vs programados
if 'Ingresos urgentes' in hospital_service_df.columns and 'Ingresos programados' in hospital_service_df.columns:
    hospital_service_df['total_ingresos_calculado'] = hospital_service_df['Ingresos urgentes'] + hospital_service_df['Ingresos programados']
    hospital_service_df['ratio_urgentes_programados'] = np.where(
        hospital_service_df['Ingresos programados'] > 0,
        hospital_service_df['Ingresos urgentes'] / hospital_service_df['Ingresos programados'],
        0
    )

# 4.4 Productividad de camas (ingresos por cama)
if 'Total ingresos' in hospital_service_df.columns and 'Camas instaladas' in hospital_service_df.columns:
    hospital_service_df['productividad_camas'] = np.where(
        hospital_service_df['Camas instaladas'] > 0,
        hospital_service_df['Total ingresos'] / hospital_service_df['Camas instaladas'],
        0
    )

# 4.5 Eficiencia de estancia (menor estancia = más eficiente, pero normalizado)
if 'Estancia media global' in hospital_service_df.columns:
    hospital_service_df['eficiencia_estancia'] = np.where(
        hospital_service_df['Estancia media global'] > 0,
        1 / hospital_service_df['Estancia media global'],  # Invertir para que menor estancia = mayor eficiencia
        0
    )

# 5. CALCULAR ÍNDICE COMPUESTO DE NIVEL DE SERVICIO

# Actualizar indicadores disponibles
service_indicators = ['tasa_ocupacion_camas', 'tasa_ingreso_urgencias', 'ratio_urgentes_programados',
                     'productividad_camas', 'eficiencia_estancia']

# Crear copia para normalización
hospital_service_normalized = hospital_service_df.copy()

for indicator in service_indicators:
    if indicator in hospital_service_normalized.columns:
        # Usar percentiles para normalizar (0-100)
        hospital_service_normalized[f'{indicator}_percentile'] = hospital_service_normalized[indicator].rank(pct=True) * 100

        # Para tasa de ocupación, el óptimo está cerca del 80-85%, no al máximo
        if indicator == 'tasa_ocupacion_camas':
            # Penalizar ocupaciones muy bajas (<50%) y muy altas (>95%)
            hospital_service_normalized[f'{indicator}_score'] = np.where(
                hospital_service_normalized[indicator] < 0.5,
                hospital_service_normalized[indicator] * 2 * 100,  # Escala 0-50 para <50%
                np.where(
                    hospital_service_normalized[indicator] > 0.95,
                    100 - (hospital_service_normalized[indicator] - 0.95) * 1000,  # Penalizar >95%
                    100  # Óptimo entre 50-95%
                )
            )
        else:
            hospital_service_normalized[f'{indicator}_score'] = hospital_service_normalized[f'{indicator}_percentile']

# Calcular índice compuesto de nivel de servicio (promedio de scores disponibles)
score_columns = [col for col in hospital_service_normalized.columns if col.endswith('_score')]
if score_columns:
    hospital_service_normalized['nivel_servicio_score'] = hospital_service_normalized[score_columns].mean(axis=1, skipna=True)
else:
    hospital_service_normalized['nivel_servicio_score'] = 0

# 6. AGREGAR INFORMACIÓN DE MUNICIPIO Y NOMBRES DESCRIPTIVOS (SOLO HOSPITALES CON MUNICIPIO)
hospital_service_final = hospital_service_normalized.merge(
    hospital_municipality_df[['hospital_id', 'municipality_id']],
    on='hospital_id',
    how='inner'  # Cambio a INNER JOIN - solo hospitales con municipio asignado
)

# 6.1 Agregar nombres de hospitales
hospital_df['id'] = hospital_df['id'].astype(str)
hospital_service_final = hospital_service_final.merge(
    hospital_df[['id', 'name']].rename(columns={'id': 'hospital_id', 'name': 'hospital_name'}),
    on='hospital_id',
    how='inner'  # INNER JOIN para asegurar que tenemos nombres
)

# 6.2 Agregar nombres de municipios y datos demográficos
municipality_df['id'] = municipality_df['id'].astype(str)
hospital_service_final = hospital_service_final.merge(
    municipality_df[['id', 'name', 'id_secondary']].rename(columns={'id': 'municipality_id', 'name': 'municipality_name'}),
    on='municipality_id',
    how='inner'  # INNER JOIN para asegurar que tenemos nombres de municipio
)

# === ANÁLISIS DE POBLACIÓN ASIGNADA A HOSPITALES ===

print("\n=== ANÁLISIS DE POBLACIÓN ASIGNADA A HOSPITALES ===\n")

# 6.3 Preparar datos demográficos (población total por municipio)
population_data = municipality_demographics_2022[
    (municipality_demographics_2022['range'] == 'total')
].copy()

print('Datos de población total por municipio:')
display(population_data[['id_secondary_municipality', 'total']].head())

# 6.4 Agregar población total del municipio a cada hospital
# Asegurar que los tipos de datos sean compatibles para el merge
municipality_df['id_secondary'] = municipality_df['id_secondary'].astype(str)
population_data['id_secondary_municipality'] = population_data['id_secondary_municipality'].astype(str)

print(f"Verificando tipos de datos antes del merge:")
print(f"hospital_service_final['id_secondary'] dtype: {hospital_service_final['id_secondary'].dtype}")
print(f"population_data['id_secondary_municipality'] dtype: {population_data['id_secondary_municipality'].dtype}")

# Verificar valores únicos para debuggear
print(f"Valores únicos en hospital_service_final['id_secondary']: {hospital_service_final['id_secondary'].nunique()}")
print(f"Valores únicos en population_data['id_secondary_municipality']: {population_data['id_secondary_municipality'].nunique()}")

# Hacer el merge con manejo de errores
try:
    hospital_service_final = hospital_service_final.merge(
        population_data[['id_secondary_municipality', 'total']].rename(columns={
            'id_secondary_municipality': 'id_secondary',
            'total': 'poblacion_total_municipio'
        }),
        on='id_secondary',
        how='left'
    )
    print(f"✅ Merge exitoso. Registros después del merge: {len(hospital_service_final)}")

except Exception as e:
    print(f"❌ Error en merge: {e}")
    print("Intentando merge alternativo...")

    # Método alternativo: crear el merge paso a paso
    population_renamed = population_data[['id_secondary_municipality', 'total']].copy()
    population_renamed = population_renamed.rename(columns={
        'id_secondary_municipality': 'id_secondary',
        'total': 'poblacion_total_municipio'
    })

    # Asegurar tipos compatibles
    population_renamed['id_secondary'] = population_renamed['id_secondary'].astype(str)
    hospital_service_final['id_secondary'] = hospital_service_final['id_secondary'].astype(str)

    # Intentar merge nuevamente
    hospital_service_final = hospital_service_final.merge(
        population_renamed,
        on='id_secondary',
        how='left'
    )
    print(f"✅ Merge alternativo exitoso. Registros: {len(hospital_service_final)}")

# Rellenar valores nulos de población con 0
hospital_service_final['poblacion_total_municipio'] = hospital_service_final['poblacion_total_municipio'].fillna(0)

# Verificar el resultado
print(f"Hospitales con población > 0: {(hospital_service_final['poblacion_total_municipio'] > 0).sum()}")
print(f"Hospitales con población = 0: {(hospital_service_final['poblacion_total_municipio'] == 0).sum()}")

# === CÁLCULO DE INDICADORES DE CAPACIDAD POR POBLACIÓN ===

print('\n=== CALCULANDO INDICADORES DE CAPACIDAD POR POBLACIÓN ===')

# 6.5 Calcular indicadores de capacidad por población
if 'Camas instaladas' in hospital_service_final.columns:
    # Camas por cada 1000 habitantes
    hospital_service_final['camas_por_1000_hab'] = np.where(
        hospital_service_final['poblacion_total_municipio'] > 0,
        (hospital_service_final['Camas instaladas'] / hospital_service_final['poblacion_total_municipio']) * 1000,
        0
    )

if 'Total ingresos' in hospital_service_final.columns:
    # Ingresos hospitalarios por cada 1000 habitantes
    hospital_service_final['ingresos_por_1000_hab'] = np.where(
        hospital_service_final['poblacion_total_municipio'] > 0,
        (hospital_service_final['Total ingresos'] / hospital_service_final['poblacion_total_municipio']) * 1000,
        0
    )

if 'Urgencias totales' in hospital_service_final.columns:
    # Urgencias por cada 1000 habitantes
    hospital_service_final['urgencias_por_1000_hab'] = np.where(
        hospital_service_final['poblacion_total_municipio'] > 0,
        (hospital_service_final['Urgencias totales'] / hospital_service_final['poblacion_total_municipio']) * 1000,
        0
    )

# 6.6 Calcular población total asignada a cada hospital (suma de todos sus municipios)
print("\n=== CALCULANDO POBLACIÓN TOTAL ASIGNADA A CADA HOSPITAL ===")

# Obtener todos los municipios asignados a cada hospital
hospital_municipalities_population = hospital_service_final.groupby('hospital_id').agg({
    'poblacion_total_municipio': 'sum',  # Suma total de población de todos los municipios asignados
    'municipality_name': lambda x: ', '.join(x.unique()),  # Lista de municipios asignados
    'municipality_id': 'count'  # Número de municipios asignados
}).rename(columns={
    'poblacion_total_municipio': 'poblacion_total_asignada',
    'municipality_name': 'municipios_asignados',
    'municipality_id': 'num_municipios_asignados'
}).reset_index()

print("Población total asignada por hospital:")
display(hospital_municipalities_population.head())

# Unir esta información de vuelta al DataFrame principal
hospital_service_final = hospital_service_final.drop(columns=['poblacion_estimada_servida'], errors='ignore')
hospital_service_final = hospital_service_final.merge(
    hospital_municipalities_population[['hospital_id', 'poblacion_total_asignada', 'num_municipios_asignados']],
    on='hospital_id',
    how='left'
)

print(f"✅ Población total asignada calculada para {len(hospital_service_final)} hospitales")
print(f"Población total asignada promedio: {hospital_service_final['poblacion_total_asignada'].mean():.0f}")
print(f"Población total asignada máxima: {hospital_service_final['poblacion_total_asignada'].max():.0f}")
print(f"Población total asignada mínima: {hospital_service_final['poblacion_total_asignada'].min():.0f}")

# 6.7 Indicadores de preparación hospitalaria para población total asignada
if 'Camas instaladas' in hospital_service_final.columns:
    # Camas por cada 1000 habitantes de la población total asignada al hospital
    hospital_service_final['camas_por_1000_asignados'] = np.where(
        hospital_service_final['poblacion_total_asignada'] > 0,
        (hospital_service_final['Camas instaladas'] / hospital_service_final['poblacion_total_asignada']) * 1000,
        0
    )

    # Clasificación de preparación según estándares internacionales (2-4 camas por 1000 hab)
    hospital_service_final['preparacion_camas'] = np.where(
        hospital_service_final['camas_por_1000_asignados'] >= 4, 'Excelente',
        np.where(hospital_service_final['camas_por_1000_asignados'] >= 2, 'Adecuada',
                np.where(hospital_service_final['camas_por_1000_asignados'] >= 1, 'Básica', 'Insuficiente'))
    )

if 'Total ingresos' in hospital_service_final.columns:
    # Capacidad de atención por población total asignada
    hospital_service_final['capacidad_atencion_por_1000_asignados'] = np.where(
        hospital_service_final['poblacion_total_asignada'] > 0,
        (hospital_service_final['Total ingresos'] / hospital_service_final['poblacion_total_asignada']) * 1000,
        0
    )

if 'Urgencias totales' in hospital_service_final.columns:
    # Urgencias por cada 1000 habitantes de población total asignada
    hospital_service_final['urgencias_por_1000_asignados'] = np.where(
        hospital_service_final['poblacion_total_asignada'] > 0,
        (hospital_service_final['Urgencias totales'] / hospital_service_final['poblacion_total_asignada']) * 1000,
        0
    )

# 6.8 Índice compuesto de preparación hospitalaria (actualizado)
preparacion_indicators = []
if 'camas_por_1000_asignados' in hospital_service_final.columns:
    preparacion_indicators.append('camas_por_1000_asignados')
if 'capacidad_atencion_por_1000_asignados' in hospital_service_final.columns:
    preparacion_indicators.append('capacidad_atencion_por_1000_asignados')
if 'urgencias_por_1000_asignados' in hospital_service_final.columns:
    preparacion_indicators.append('urgencias_por_1000_asignados')

# Normalizar indicadores de preparación usando percentiles
for indicator in preparacion_indicators:
    if indicator in hospital_service_final.columns:
        hospital_service_final[f'{indicator}_percentile'] = hospital_service_final[indicator].rank(pct=True) * 100

# Calcular índice de preparación poblacional actualizado
if preparacion_indicators:
    percentile_cols = [f'{ind}_percentile' for ind in preparacion_indicators if f'{ind}_percentile' in hospital_service_final.columns]
    if percentile_cols:
        hospital_service_final['indice_preparacion_poblacional'] = hospital_service_final[percentile_cols].mean(axis=1, skipna=True)
    else:
        hospital_service_final['indice_preparacion_poblacional'] = 0
else:
    hospital_service_final['indice_preparacion_poblacional'] = 0

# === MOSTRAR RESULTADOS DEL ANÁLISIS POBLACIONAL ACTUALIZADO ===

print('\n=== INDICADORES DE PREPARACIÓN HOSPITALARIA POR POBLACIÓN TOTAL ASIGNADA ===')
population_columns = ['hospital_name', 'num_municipios_asignados', 'poblacion_total_asignada']
if 'Camas instaladas' in hospital_service_final.columns:
    population_columns.append('Camas instaladas')
if 'camas_por_1000_asignados' in hospital_service_final.columns:
    population_columns.extend(['camas_por_1000_asignados', 'preparacion_camas'])
if 'capacidad_atencion_por_1000_asignados' in hospital_service_final.columns:
    population_columns.append('capacidad_atencion_por_1000_asignados')
if 'urgencias_por_1000_asignados' in hospital_service_final.columns:
    population_columns.append('urgencias_por_1000_asignados')
population_columns.append('indice_preparacion_poblacional')

# Eliminar duplicados por hospital para mostrar resultados únicos
hospital_unique_results = hospital_service_final.drop_duplicates(subset=['hospital_id'])
population_results = hospital_unique_results[population_columns].round(4)
display(population_results.head(10))

print('\n=== TOP 10 HOSPITALES MEJOR PREPARADOS PARA SU POBLACIÓN TOTAL ASIGNADA ===')
top_prepared = hospital_unique_results.nlargest(10, 'indice_preparacion_poblacional')[population_columns]
display(top_prepared)

print('\n=== HOSPITALES CON MENOR PREPARACIÓN PARA SU POBLACIÓN TOTAL ASIGNADA ===')
least_prepared = hospital_unique_results.nsmallest(10, 'indice_preparacion_poblacional')[population_columns]
display(least_prepared)

# === ESTADÍSTICAS DE PREPARACIÓN POBLACIONAL ACTUALIZADAS ===
print('\n=== ESTADÍSTICAS DE PREPARACIÓN POBLACIONAL (POBLACIÓN TOTAL ASIGNADA) ===')

if 'camas_por_1000_asignados' in hospital_unique_results.columns:
    print(f"\n🏥 ANÁLISIS DE CAMAS POR POBLACIÓN TOTAL ASIGNADA:")
    print(f"   • Promedio de camas por 1000 habitantes asignados: {hospital_unique_results['camas_por_1000_asignados'].mean():.2f}")
    print(f"   • Mediana: {hospital_unique_results['camas_por_1000_asignados'].median():.2f}")
    print(f"   • Máximo: {hospital_unique_results['camas_por_1000_asignados'].max():.2f}")
    print(f"   • Mínimo: {hospital_unique_results['camas_por_1000_asignados'].min():.2f}")

    # Distribución por clasificación de preparación
    if 'preparacion_camas' in hospital_unique_results.columns:
        preparacion_counts = hospital_unique_results['preparacion_camas'].value_counts()
        total_hospitals = len(hospital_unique_results)
        print(f"\n📊 DISTRIBUCIÓN DE PREPARACIÓN DE CAMAS:")
        for categoria, count in preparacion_counts.items():
            print(f"   • {categoria}: {count} hospitales ({count/total_hospitals*100:.1f}%)")

if 'capacidad_atencion_por_1000_asignados' in hospital_unique_results.columns:
    print(f"\n🚑 ANÁLISIS DE CAPACIDAD DE ATENCIÓN POR POBLACIÓN ASIGNADA:")
    print(f"   • Promedio de ingresos por 1000 habitantes asignados: {hospital_unique_results['capacidad_atencion_por_1000_asignados'].mean():.2f}")
    print(f"   • Mediana: {hospital_unique_results['capacidad_atencion_por_1000_asignados'].median():.2f}")

if 'urgencias_por_1000_asignados' in hospital_unique_results.columns:
    print(f"\n🚨 ANÁLISIS DE URGENCIAS POR POBLACIÓN ASIGNADA:")
    print(f"   • Promedio de urgencias por 1000 habitantes asignados: {hospital_unique_results['urgencias_por_1000_asignados'].mean():.2f}")
    print(f"   • Mediana: {hospital_unique_results['urgencias_por_1000_asignados'].median():.2f}")

print(f"\n📈 ÍNDICE DE PREPARACIÓN POBLACIONAL (ACTUALIZADO):")
print(f"   • Promedio: {hospital_unique_results['indice_preparacion_poblacional'].mean():.2f}")
print(f"   • Desviación estándar: {hospital_unique_results['indice_preparacion_poblacional'].std():.2f}")

# === ANÁLISIS DETALLADO POR HOSPITAL CON MÚLTIPLES MUNICIPIOS ===
print('\n=== HOSPITALES CON MÚLTIPLES MUNICIPIOS ASIGNADOS ===')

multi_municipality_hospitals = hospital_unique_results[hospital_unique_results['num_municipios_asignados'] > 1]
if len(multi_municipality_hospitals) > 0:
    print(f"Hospitales que sirven múltiples municipios: {len(multi_municipality_hospitals)}")
    multi_cols = ['hospital_name', 'num_municipios_asignados', 'poblacion_total_asignada', 'camas_por_1000_asignados', 'preparacion_camas']
    available_multi_cols = [col for col in multi_cols if col in multi_municipality_hospitals.columns]
    display(multi_municipality_hospitals[available_multi_cols].sort_values('poblacion_total_asignada', ascending=False))
else:
    print("No hay hospitales que sirvan múltiples municipios en el análisis.")

# === RESUMEN EJECUTIVO ===
print('\n' + '='*80)
print('📋 RESUMEN EJECUTIVO - PREPARACIÓN HOSPITALARIA PARA POBLACIÓN ASIGNADA')
print('='*80)

total_hospitals = len(hospital_unique_results)
total_population_served = hospital_unique_results['poblacion_total_asignada'].sum()
total_beds = hospital_unique_results['Camas instaladas'].sum() if 'Camas instaladas' in hospital_unique_results.columns else 0

print(f"\n🏥 OVERVIEW GENERAL:")
print(f"   • Total de hospitales analizados: {total_hospitals}")
print(f"   • Población total asignada: {total_population_served:,.0f} habitantes")
print(f"   • Total de camas disponibles: {total_beds:,.0f}")
if total_population_served > 0:
    print(f"   • Ratio general camas/población: {(total_beds/total_population_served)*1000:.2f} camas por 1000 hab")

if 'preparacion_camas' in hospital_unique_results.columns:
    print(f"\n📊 DISTRIBUCIÓN DE PREPARACIÓN:")
    preparacion_summary = hospital_unique_results['preparacion_camas'].value_counts()
    for nivel in ['Excelente', 'Adecuada', 'Básica', 'Insuficiente']:
        count = preparacion_summary.get(nivel, 0)
        pct = (count/total_hospitals)*100
        print(f"   • {nivel}: {count} hospitales ({pct:.1f}%)")

# Hospitales críticos (preparación insuficiente y alta población)
if 'preparacion_camas' in hospital_unique_results.columns:
    critical_hospitals = hospital_unique_results[
        (hospital_unique_results['preparacion_camas'] == 'Insuficiente') &
        (hospital_unique_results['poblacion_total_asignada'] > hospital_unique_results['poblacion_total_asignada'].median())
    ]

    if len(critical_hospitals) > 0:
        print(f"\n⚠️  HOSPITALES EN SITUACIÓN CRÍTICA:")
        print(f"   • {len(critical_hospitals)} hospitales con preparación insuficiente y alta población asignada")
        critical_cols = ['hospital_name', 'poblacion_total_asignada', 'camas_por_1000_asignados']
        display(critical_hospitals[critical_cols].sort_values('poblacion_total_asignada', ascending=False))

print('\n' + '='*80)


Hospitales:


,id,name_large,name,id_municipality,utm_x,utm_y,tag,address,url,id_via,name_municipality,latitude,longitude
0,280724,Hospital Centro Sanitario de Vida y Esperanza....,HOSPITAL CENTRO SANITARIO DE VIDA Y ESPERANZA,28079,436745,4471110,Hospital Centro Sanitario de Vida y Esperanza,"Glorieta del Ejército, s/n",https://centrossanitarios.sanidadmadrid.org/Re...,07901901,Madrid,40.388185,-3.745275
1,CH0001,Fremap Hospital y Centro de Rehabilitación de ...,FREMAP HOSPITAL Y CENTRO DE REHABILITACIÓN DE ...,28080,427799,4478345,Fremap Hospital y Centro de Rehabilitación de ...,"Carretera de Pozuelo, 61",https://www.fremap.es,08000170,Majadahonda,40.452632,-3.851488
2,CH0004,Centro San Juan de Dios. Avenida de San Juan d...,CENTRO SAN JUAN DE DIOS,28040,447682,4445737,Centro San Juan de Dios,"Avenida de San Juan de Dios, 1",https://sjd.es/location/centro-san-juan-de-dio...,040A0157,Ciempozuelos,40.160362,-3.614350
3,CH0006,Fuensanta S.l. (Clínica Fuensanta). Calle de A...,FUENSANTA S.L. (CLINICA FUENSANTA),28079,445713,4476806,Fuensanta S.l. (Clínica Fuensanta),"Calle de Arturo Soria, 17",https://www.viamedsalud.com/hospital-fuensanta/,07900582,Madrid,40.440131,-3.640106
4,CH0008,"Clínica La Luz, S.l.. Calle del General Rodrig...","CLINICA LA LUZ, S.L.",28079,439451,4477308,"Clínica La Luz, S.l.","Calle del General Rodrigo, 8",https://www.quironsalud.com/es/red-centros/hos...,07909065,Madrid,40.444221,-3.713985


Actividad hospitalaria:


,hospital_id,year,type_activity,total
0,CH0096,2012,Estancia media global,5.72
1,CH0096,2013,Estancia media global,6.08
2,CH0096,2014,Estancia media global,6.03
3,CH0096,2015,Estancia media global,6.22
4,CH0096,2016,Estancia media global,6.16


Hospital-Municipio:


...

,hospital_name,num_municipios_asignados,poblacion_total_asignada,Camas instaladas,camas_por_1000_asignados,preparacion_camas,capacidad_atencion_por_1000_asignados,urgencias_por_1000_asignados,indice_preparacion_poblacional
137,HOSPITAL UNIVERSITARIO INFANTA LEONOR,1,3267027,361.0,0.110498,Insuficiente,5.253400,55.981172,0.793651
35,HHOSPITAL CLINICO SAN CARLOS,1,3267027,861.0,0.263542,Insuficiente,8.786276,44.633546,0.992063
0,HOSPITAL DE EL ESCORIAL,17,152371,92.0,0.603789,Insuficiente,20.345079,342.315795,7.341270
31,HOSPITAL UNIVERSITARIO SEVERO OCHOA,4,403522,386.0,0.956577,Insuficiente,37.467598,299.485530,23.908730
116,HOSPITAL UNIVERSITARIO DEL SURESTE,21,207527,132.0,0.636062,Insuficiente,40.341739,594.476863,34.126984
51,HOSPITAL UNIVERSITARIO INFANTA SOFIA,52,341322,276.0,0.808621,Insuficiente,49.961620,420.810847,37.400794
17,HOSPITAL UNIVERSITARIO DE MOSTOLES,2,243484,328.0,1.347111,Básica,49.259910,449.109592,49.702381
143,HOSPITAL REY JUAN CARLOS,18,328893,362.0,1.100662,Básica,63.698528,499.414703,63.591270
103,HOSPITAL UNIVERSITARIO INFANTA CRISTINA,8,172396,188.0,1.090513,Básica,51.393304,663.466670,67.956349
46,HOSPITAL UNIVERSITARIO HENARES,5,165087,263.0,1.593099,Básica,68.194346,608.860782,77.182540



=== ESTADÍSTICAS DE PREPARACIÓN POBLACIONAL (POBLACIÓN TOTAL ASIGNADA) ===

🏥 ANÁLISIS DE CAMAS POR POBLACIÓN TOTAL ASIGNADA:
   • Promedio de camas por 1000 habitantes asignados: 1.29
   • Mediana: 1.32
   • Máximo: 2.34
   • Mínimo: 0.11

📊 DISTRIBUCIÓN DE PREPARACIÓN DE CAMAS:
   • Básica: 9 hospitales (50.0%)
   • Insuficiente: 6 hospitales (33.3%)
   • Adecuada: 3 hospitales (16.7%)

🚑 ANÁLISIS DE CAPACIDAD DE ATENCIÓN POR POBLACIÓN ASIGNADA:
   • Promedio de ingresos por 1000 habitantes asignados: 57.50
   • Mediana: 60.50

🚨 ANÁLISIS DE URGENCIAS POR POBLACIÓN ASIGNADA:
   • Promedio de urgencias por 1000 habitantes asignados: 578.97
   • Mediana: 601.67

📈 ÍNDICE DE PREPARACIÓN POBLACIONAL (ACTUALIZADO):
   • Promedio: 59.01
   • Desviación estándar: 33.39

=== HOSPITALES CON MÚLTIPLES MUNICIPIOS ASIGNADOS ===
Hospitales que sirven múltiples municipios: 15


,hospital_name,num_municipios_asignados,poblacion_total_asignada,camas_por_1000_asignados,preparacion_camas
31,HOSPITAL UNIVERSITARIO SEVERO OCHOA,4,403522,0.956577,Insuficiente
51,HOSPITAL UNIVERSITARIO INFANTA SOFIA,52,341322,0.808621,Insuficiente
143,HOSPITAL REY JUAN CARLOS,18,328893,1.100662,Básica
19,HOSPITAL UNIVERSITARIO PRINCIPE DE ASTURIAS,12,251343,2.017164,Adecuada
17,HOSPITAL UNIVERSITARIO DE MOSTOLES,2,243484,1.347111,Básica
36,HOSPITAL UNIVERSITARIO DE GETAFE,2,239268,2.269422,Adecuada
39,HOSPITAL UNIVERSITARIO FUENLABRADA,3,215400,1.917363,Básica
116,HOSPITAL UNIVERSITARIO DEL SURESTE,21,207527,0.636062,Insuficiente
103,HOSPITAL UNIVERSITARIO INFANTA CRISTINA,8,172396,1.090513,Básica
46,HOSPITAL UNIVERSITARIO HENARES,5,165087,1.593099,Básica



📋 RESUMEN EJECUTIVO - PREPARACIÓN HOSPITALARIA PARA POBLACIÓN ASIGNADA

🏥 OVERVIEW GENERAL:
   • Total de hospitales analizados: 18
   • Población total asignada: 9,903,453 habitantes
   • Total de camas disponibles: 5,870
   • Ratio general camas/población: 0.59 camas por 1000 hab

📊 DISTRIBUCIÓN DE PREPARACIÓN:
   • Excelente: 0 hospitales (0.0%)
   • Adecuada: 3 hospitales (16.7%)
   • Básica: 9 hospitales (50.0%)
   • Insuficiente: 6 hospitales (33.3%)

⚠️  HOSPITALES EN SITUACIÓN CRÍTICA:
   • 4 hospitales con preparación insuficiente y alta población asignada


,hospital_name,poblacion_total_asignada,camas_por_1000_asignados
35,HHOSPITAL CLINICO SAN CARLOS,3267027,0.263542
137,HOSPITAL UNIVERSITARIO INFANTA LEONOR,3267027,0.110498
31,HOSPITAL UNIVERSITARIO SEVERO OCHOA,403522,0.956577
51,HOSPITAL UNIVERSITARIO INFANTA SOFIA,341322,0.808621
